In [ ]:

Projects
 Candace Peters - BINF 5503: Data Analytics and Storytelling


Help
C
Account
$16.40
10
0
2025-03-14-file-1.ipynb


Tabs
Explorer
New
Log
Find
Servers
Users
Upgrades
Processes
Settings
 Server
 
 

Python 3 (system-wide)
Trusted
Kernel is idle (halt...)
Code
CPU
RAM
8.58s
1
In [3]:
Age
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from lifelines import CoxPHFitter
from lifelines.datasets import load_rossi
from lifelines import KaplanMeierFitter
from sksurv.linear_model import CoxnetSurvivalAnalysis
from sklearn.model_selection import train_test_split
from sksurv.metrics import concordance_index_censored
from sksurv.ensemble import RandomSurvivalForest
from sksurv.datasets import load_whas500
from sksurv.preprocessing import OneHotEncoder
from sklearn.inspection import permutation_importance
from lifelines.statistics import logrank_test
from sksurv.util import Surv
​
# Load the RADCURE_Clinical Dataset
data = pd.read_excel("RADCURE_Clinical_v04_20241219.xlsx")
​
# Inspect the Dataset
print(data.head())
print(data.info())
print(data.describe())
​
# Kaplan-Meier Analysis
# Fit the Kaplan-Meier estimator
​
kmf = KaplanMeierFitter()
plt.figure(figsize=(10, 6))
​
# Define the groups based on a column, e.g., TreatmentType (modify as needed)
group_col = "Chemo"
unique_groups = data[group_col].unique()
​
# Loop over unique groups for Kaplan-Meier analysis
for group in unique_groups:
    group_data = data[data[group_col] == group]
    
    # Fit and plot survival curves
    kmf.fit(group_data['time'], event_observed=group_data['event'])
    kmf.plot_survival_function(label=f"{group_col} = {group}")
​
plt.title(f'Kaplan-Meier Curve for {group_col}')
plt.xlabel('Age)')
plt.ylabel('Survival Probability')
plt.legend()
plt.grid()
plt.show()
​
# Log-rank test for statistical comparison
group1 = data[data[group_col] == unique_groups[0]]
group2 = data[data[group_col] == unique_groups[1]]
​
logrank_result = logrank_test(group1["Age"], group2["Chemo"], 
                              event_observed_A=group1["event"], 
                              event_observed_B=group2["event"])
​
print(f"Comparison: {group_col} {unique_groups[0]} vs {group_col} {unique_groups[1]}")
print(f"Log-rank test p-value: {logrank_result.p_value:.4f}")
​
# Cox Proportional Hazards Regression
# Convert event column to Boolean
data["event"] = data["event"].astype(bool)
​
# Select covariates for Cox regression (modify as needed)
covariates = ["Age", "TumorStage", "TreatmentType"]
​
# Convert categorical variables to numeric (if necessary)
for col in covariates:
    if data[col].dtype == 'object' or data[col].dtype.name == 'category':
        data[col] = data[col].astype("category").cat.codes
​
# Fit the Cox proportional hazards model
cph = CoxPHFitter()
cph.fit(data[["Age", "Chemo"] + covariates], duration_col="Age", event_col="Chemo")
​
# Print the summary of the model
cph.print_summary()
​
# Plot the coefficients
plt.figure(figsize=(8, 6))
cph.plot()
plt.title('Cox Regression Coefficients')
plt.show()
​
# Validate proportional hazards assumption
ph_test_results = cph.check_assumptions(data, p_value_threshold=0.05)
​
# Random Survival Forests (RSF)
# Encode categorical variables using OneHotEncoder
encoder = OneHotEncoder(drop="first", sparse=False)
encoded_features = encoder.fit_transform(data[covariates])
​
# Convert encoded features to a DataFrame
encoded_feature_names = encoder.get_feature_names_out(covariates)
encoded_df = pd.DataFrame(encoded_features, columns=encoded_feature_names, index=data.index)
​
# Replace original categorical columns with encoded values
data_x = pd.concat([data.drop(columns=covariates), encoded_df], axis=1)
​
# Prepare survival data in structured format for sksurv
data_y = Surv.from_dataframe(event="event", Age="Age", data=data)
​
# Train the Random Survival Forest model
rsf = RandomSurvivalForest(n_estimators=100, min_samples_split=10, min_samples_leaf=5, random_state=42)
rsf.fit(data_x, data_y)
​
# Compute Concordance Index for RSF
rsf_cindex = rsf.score(data_x, data_y)
print(f"Random Survival Forest C-index: {rsf_cindex:.4f}")
​
# Compute Concordance Index for Cox Regression (for comparison)
cph_cindex = cph.concordance_index_
print(f"Cox Regression C-index: {cph_cindex:.4f}")
​
# Perform Permutation Feature Importance Analysis
result = permutation_importance(rsf, data_x, data_y, n_repeats=15, random_state=42)
feature_importance = pd.DataFrame(
    {
        "importances_mean": result["importances_mean"],
        "importances_std": result["importances_std"],
    },
    index=data_x.columns,
).sort_values(by="importances_mean", ascending=False)
​
# Plot Feature Importance
plt.figure(figsize=(10, 6))
plt.title("Feature Importances in Random Survival Forest")
plt.barh(feature_importance.index, feature_importance["importances_mean"], 
         xerr=feature_importance["importances_std"], align="center", color="teal")
plt.xlabel("Mean Importance Score")
plt.ylabel("Features")
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()
​